### GFP_AEQV processing

In [2]:
import evoVAE.utils.seq_tools as st
from evoVAE.utils.webservice import get_sequence_batch
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", None)
from ete3 import Tree
from sequence import *

# Reading in alignment 

In [2]:
aln = st.read_aln_file('../data/dms_data/GFP_AEQVI_full_04-29-2022_b08.a2m')
#st.write_fasta_file(f"../data/alignments/gfp_alns/GFP_AEQVI_full_04-29-2022_b08.aln", aln)

#draws = []
#for draw in range(100):
#    draws.append(aln.sample(n=250, random_state=draw, replace=False))

#for i, seqs in enumerate(draws):
#    st.write_fasta_file(f"../data/alignments/gfp_alns/GFP_AEQVI_full_04-29-2022_b08_rand_subset_{i}.aln", seqs)
#aln

Reading the alignment: ../data/dms_data/GFP_AEQVI_full_04-29-2022_b08.a2m
Checking for bad characters: ['B', 'J', 'X', 'Z']
Removed 16 sequences
Performing one hot encoding
Number of seqs: 380


# Collecting annotations about sequences

In [3]:
ids = aln['id']
ids = [x for x in ids.apply(lambda x: x.split('/')[0]).values]

print( [x.split('_')[1] for x in ids])
# Sequences found in the very distant branch of the tree. 
outgroup = ["UniRef100_Q6RYS7/1-232",
"UniRef100_UPI000B5ABD49/2-234",
"UniRef100_A0A5J6CYI5/1-229",
"UniRef100_Q6RYS6/40-259",
"UniRef100_Q6RYS5/2-223",
"UniRef100_J9PIH6/6-230",
"UniRef100_J9PJD5/7-231",
"UniRef100_A0A7M5TYI8/7-231",
"UniRef100_A0A5J6CYV7/1-229",
"UniRef100_A0A5J6CYI7/1-229",
"UniRef100_A0A5J6CYT5/1-229",
"UniRef100_A0A5J6CYV4/1-229",
"UniRef100_A0A5J6CYK8/1-229",
"UniRef100_A0A7M5TUN8/3-229",
"UniRef100_A0A7M5UKF9/3-229",
"UniRef100_A0A7M5WQI9/3-229",
"UniRef100_A0A7M5UPI8/3-229",
"UniRef100_A0A7M5V224/3-229",
"UniRef100_J9PGT0/3-229",
"UniRef100_A0A7M5WWU1/3-229",
"UniRef100_A0A7M5XGN4/3-229",
"UniRef100_G1JSF2/3-231",
"UniRef100_G1JSF3/3-231",
"UniRef100_G1JSF4/3-231",
"UniRef100_A0A7M5WWD0/7-234",
"UniRef100_A0A7M5V407/7-234",
"UniRef100_A0A7M6DMT2/54-281",
"UniRef100_J9PGG2/7-234",
"UniRef100_A0A7M5X361/50-277",
"UniRef100_D7PM05/5-232",
"UniRef100_D7PM10/5-232",
"UniRef100_D7PM12/5-232",
"UniRef100_D7PM04/5-232",
"UniRef100_D7PM06/5-232",]

ID = 0 
SPECIES = 1
# in-group sequences 
outgroup = [x.split('/')[ID] for x in outgroup]
print(len(outgroup))
# FIX ME, querying for uniref seqs is still broken, skipping first uniprot entry 
ingroup = [x for x in ids[1:] if x not in outgroup]
print(len(ingroup))

uniprot_cols =  ["id", "organism"]
outgroup_query = get_sequence_batch(outgroup, uniprot_cols, query_field = "id", database="uniref")
ingroup_query = get_sequence_batch(ingroup, uniprot_cols, query_field = "id", database="uniref")

outgroup_species = pd.Series([x[SPECIES] for x in outgroup_query]).value_counts()
ingroup_species = pd.Series([x[SPECIES] for x in ingroup_query]).value_counts()

['AEQVI', 'UPI0011C34247', 'UPI0011C34247', 'UPI0011C3426C', 'UPI0011C3426C', 'UPI001C2E920B', 'UPI0000EA3547', 'UPI000B5ABD33', 'UPI000B5ABDBE', 'UPI000B5ABD69', 'UPI000B5ABD53', 'UPI0009795733', 'UPI00135F1855', 'P42212', 'W6KDG8', 'UPI000B5ABDBC', 'UPI0001986815', 'A0A2V2QXC3', 'UPI0002A54BBC', 'Q8GHE4', 'UPI000B5ABD1A', 'UPI000B5ABD2E', 'UPI000067C63B', 'A0A8E6TY54', 'UPI000B5ABD28', 'A0A6S7E9W3', 'UPI000B5DBA18', 'UPI000B5ABD5D', 'A0A5P9VSM6', 'A0A0U3A3D8', 'UPI001C63CC57', 'UPI0001D1F7C5', 'UPI001D0CF09B', 'UPI000C173AFD', 'A0A1V0D974', 'A0A7G8ZY66', 'UPI000B5ABD97', 'UPI00157E3332', 'UPI000B5ABD72', 'UPI001CB8BAD1', 'UPI000B5ABDB8', 'UPI000B8BB2ED', 'UPI0001AB2972', 'UPI00001118CE', 'UPI000B5ABD8B', 'A0A6N6XNT4', 'A0A193CK14', 'UPI000B5ABD25', 'UPI000B5ABD67', 'UPI0018D5A8B2', 'UPI0006891DAB', 'J8VIQ3', 'UPI0010706A2B', 'UPI000F7351CB', 'UPI00157A33EC', 'UPI00027B2CE5', 'UPI0001AB2973', 'UPI000DF72107', 'A0A6M3ZKY0', 'UPI0001986814', 'A0A193DTF7', 'A0A059PIQ0', 'UPI0010A1F85E', 

# Removing synthetic sequences

In [4]:
synthetic_seqs = [x for x,y in ingroup_query if y == 'synthetic construct']

no_syn_aln_seqs = []
all_seqs = []
for name, seq in zip(aln['id'], aln['sequence']):

    if name.split('/')[0] not in synthetic_seqs:
        x = Sequence(seq, name=name, gappy=True)
        no_syn_aln_seqs.append(x)

    x = Sequence(seq, name=name, gappy=True)
    all_seqs.append(x)

print(len(no_syn_aln_seqs), len(all_seqs))
writeFastaFile(filename='no_synthetic_GFP_AEQVI_full_04-29-2022_b08.aln', seqs=no_syn_aln_seqs)

360 380


In [5]:
all_aln = Alignment(all_seqs)
no_syn_aln = Alignment(no_syn_aln_seqs)

con_all = all_aln.getConsensus() 
no_syn_con = no_syn_aln.getConsensus()
count = 0 
for x, y in zip(no_syn_con, con_all):
    
    if x != y:
        print(f"Difference at : {count}")

    count += 1


Difference at : 162


In [6]:
outgroup_species

Clytia hemisphaerica                      16
Clytia gregaria (Gregarious jellyfish)     5
Aequorea australis                         4
Obelia sp. MH-2011                         3
Phialidium sp. SL-2003                     2
Aequorea victoria (Water jellyfish)        2
Anthomedusae sp. SL-2003                   2
Name: count, dtype: int64

In [7]:
ingroup_species

Aequorea victoria                                                                                                                                                                           189
synthetic construct                                                                                                                                                                          20
Homo sapiens                                                                                                                                                                                 14
Rattus norvegicus                                                                                                                                                                            11
Aequorea macrodactyla                                                                                                                                                                         9
Aequorea victoria (Water jellyfish)     

# Rooting tree

Looking at the run trees, the sequence UniRef100_A0A5J6CYI5/1-229 is consistently placed by itself and this will be used to help root the tree. The ancestor two nodes up will be used to place the root. 

In [73]:
# grab the node, returns a list so get first value
TREE_DIR = '../data/alignments/trees/'
for i in range(1,101):
    tree_path = TREE_DIR + f'run_{i}.treefile'
    t = Tree(tree_path)
    start_node = t.get_leaves_by_name(name="UniRef100_A0A5J6CYI5/1-229")[0]

    ancestor = start_node.up.up
    t.set_outgroup(ancestor)
    t.write(outfile=f"rooted_run_{i}.nwk")

'(((UniRef100_A0A5J6CYI5/1-229:0.170122,(((((UniRef100_A0A7M5UPI8/3-229:2.0185e-06,UniRef100_A0A7M5WQI9/3-229:0.0270395)1:0.00825733,(UniRef100_A0A7M5V224/3-229:2.1096e-06,((UniRef100_A0A7M5XGN4/3-229:0.00443338,UniRef100_A0A7M5WWU1/3-229:0.032247)1:0.0179585,UniRef100_J9PGT0/3-229:2.9211e-06)1:0.0225424)1:0.0141129)1:0.0368597,(UniRef100_A0A7M5UKF9/3-229:0.0117844,UniRef100_A0A7M5TUN8/3-229:0.164498)1:0.0366635)1:0.288264,((UniRef100_G1JSF2/3-231:0.0151093,(UniRef100_G1JSF4/3-231:0.027975,UniRef100_G1JSF3/3-231:0.0136302)1:0.0514318)1:0.282053,(((UniRef100_D7PM05/5-232:2.0139e-06,((UniRef100_D7PM12/5-232:2.0949e-06,(UniRef100_D7PM06/5-232:0.00443406,UniRef100_D7PM04/5-232:0.00443116)1:1.0091e-06)1:2.0768e-06,UniRef100_D7PM10/5-232:0.00443483)1:0.00444638)1:0.0278798,(UniRef100_A0A7M6DMT2/54-281:0.00611566,(UniRef100_A0A7M5X361/50-277:2.0402e-06,UniRef100_J9PGG2/7-234:0.00887849)1:0.0209102)1:0.0432411)1:0.0454175,(UniRef100_A0A7M5V407/7-234:0.0103643,UniRef100_A0A7M5WWD0/7-234:0.02774

'((((A0A3N1NMM4_PhoQ_UniRef90:0.636274,((((A0A935YSN4_PhoQ_UniRef90:0.538667,A0A3D4Q711_PhoQ_UniRef90:0.351407)1:0.118795,A0A4Q5V9R2_PhoQ_UniRef90:0.454298)1:0.134457,A0A1Y0G129_PhoQ_UniRef90:0.279168)1:0.118439,A0A928V440_PhoQ_UniRef90:0.510323)1:0.249848)1:0.102478,A0A9J7A4I5_PhoQ_UniRef90:1.01864)1:0.132461,((((A0A545STM0_PhoQ_UniRef90:0.586993,(Q21FC0_PhoQ_UniRef90:0.790908,A0A952ID09_PhoQ_UniRef90:0.735006)1:0.421092)1:0.107525,(A0AA37T4F5_PhoQ_UniRef90:0.735303,A0A2U3B9L0_PhoQ_UniRef90:1.17527)1:0.126644)1:0.0263925,A0A845FVC5_PhoQ_UniRef90:1.17435)1:0.12817,(((((A0A8J7JWZ1_PhoQ_UniRef90:0.420511,A0A2G6JAY7_PhoQ_UniRef90:0.571774)1:0.257319,(A0A1A9ETM3_PhoQ_UniRef90:0.527635,((((A0A063Y6P3_PhoQ_UniRef90:0.406274,W9UYP3_PhoQ_UniRef90:0.393823)1:0.368164,A0A2P8EWV8_PhoQ_UniRef90:0.543418)1:0.0658743,A0A1H6DBX6_PhoQ_UniRef90:0.430626)1:0.0736177,A0A9Q5C7B1_PhoQ_UniRef90:0.808418)1:0.158676)1:0.0727464)1:0.0340602,(A0A437QAL5_PhoQ_UniRef90:0.440854,A0A1I2M0F7_PhoQ_UniRef90:0.28293)1: